In [ ]:
import pandas as pd
nba = pd.read_csv("nba_all_elo.csv")
type(nba)

In [ ]:
print(len(nba)) # number of rows
print(nba.shape) # dimension sizes
nba.head()

In [ ]:
# display options
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)
nba.tail()

In [ ]:
# initial glimpse (type-checking, memory usage, count, null values) 
# (Try to not leave things out as Dtype being object)
nba.info()

In [ ]:
# initial glimpse (numeric summary)
nba.describe()

In [ ]:
# initial glimpse (non-numeric summary)
nba.describe(include="object")

In [ ]:
# value counts

# Number of games each team played
nba["team_id"].value_counts()

# Dates which BOS (team with most games played) played
print("Oldest, Latest Date which Boston played a game:\n", "-" * 48 + "\n",
      nba.loc[nba["team_id"] == "BOS", "date_game"].agg(("min", "max")), sep="")
# Teams Boston has played against
print("\n# of times Boston has played against each of their opponent teams\n", "-" * 65 + "\n",
      nba.loc[nba["team_id"] == "BOS", "opp_id"].value_counts(), sep="")


In [ ]:
# Do a search for Baltimore games where both teams scored over 100 points. 
# In order to see each game only once, you’ll need to exclude duplicates:
blb_special_games = nba[
    (nba["team_id"] == "BLB") & 
    (nba["pts"] > 100) &
    (nba["opp_pts"] > 100)
    ]
blb_special_games.loc[:, ("game_id", "_iscopy", "team_id", "pts", "opp_id", "opp_pts")]

,team_id,game_id,wins
0,ATL,4035,3517
1,BOS,5997,3134
2,CHI,4307,2855
3,DET,4985,2430
4,LAL,5078,2306
5,MIL,4034,2257
6,NYK,5769,2232
7,PHI,4533,2056
8,PHO,4080,2050
9,POR,3870,1994
